In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
in_usoon_priv_database = "/content/drive/My Drive/erevna/Usoon_private_database/"
out_usoon_priv_database = "/content/drive/My Drive/erevna/"'

ids=["malika","renuka","other"]
modes=["train","val"]

In [ ]:
!pip install mtcnn

!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 "shape_predictor_68_face_landmarks.dat.bz2"

1) Code that will allow company to extract out and save (in private database) only face of user A from rest all extracted faces from the pictures uploaded in his/her profile.<br>

2) To do so, for first 25 times of user A uploading his/her pic in the Usoon profile, company will be asking him/her out of all extracted photos, which one is his/her own photo..........After 25 times, algorithm by itself will extract all faces from the photos uploaded by user A and will only keep user A's face. 

3) These face cutouts of user A will be beneficial to match his/her presence in some photo uploaded by user B, which in turn if validates True will save the user A in friend list of user B.

In [ ]:
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
import os
import sys
import cv2
import shutil
import numpy as np
from numpy import savez_compressed, load, asarray, expand_dims 
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
import pickle
from matplotlib.patches import Rectangle
import random 
from PIL import Image
import dlib

# **Refining Collected Dataset**

**Extracting** faces from images present in collected dataset, **aligning** them to a similar pattern, **asking** user which face-cutout belongs to him/her and **saving** only those face-cutouts which belongs to user (according to users confirmation)

In [ ]:
#this function will align all photos into a similar pattern


def face_alignment(image,faces,i):
    dimension=list()
    #saving dimensions of "leftEyeCenter", "rightEyeCenter" and "nose" presennt inside sub-dictionary which is value of a key "keypoints" of dictionary named "faces",
    #in list named as "dimension" 
    for key in faces[i]['keypoints']:
      a=faces[i]['keypoints'][key]
      dimension.append(a)
    

    #retreiving the dimemsions from list called "dimension", into thier respective named variables "leftEyeCenter", "rightEyeCenter" and "nose" 
    left_eye_center=dimension[0]
    right_eye_center=dimension[1]
    nose=dimension[2]


    #extracting "x-coordinate" and "y-coordinate" of "leftEyeCenter" into "leftEyeCenter_x" and "leftEyeCenter_y" respectively   
    a=str(left_eye_center)
    aa=a.split("(")[1]
    left_eye_center_x=aa.split(",")[0] 
    left_eye_center_x=int(left_eye_center_x)
    aaa=aa.split(",")[1]
    aaaa=aaa.split(" ")[1]
    left_eye_center_y=aaaa.split(")")[0] 
    left_eye_center_y=int(left_eye_center_y)


    #extracting "x-coordinate" and "y-coordinate" of "rightEyeCenter" into "rightEyeCenter_x" and "rightEyeCenter_y" respectively
    b=str(right_eye_center)
    bb=b.split("(")[1]
    right_eye_center_x=bb.split(",")[0]
    right_eye_center_x=int(right_eye_center_x)
    bbb=bb.split(",")[1]
    bbbb=bbb.split(" ")[1]
    right_eye_center_y=bbbb.split(")")[0] 
    right_eye_center_y=int(right_eye_center_y)


    # find angle of line joining centres of left and right eyes, by using slop of the line formulae.
    dy = right_eye_center_y - left_eye_center_y
    dx = right_eye_center_x - left_eye_center_x
    angle = np.degrees(np.arctan2(dY, dX))


    # to get the face at the center of the image, set left eye to 35% from x-axis and 35% from y-axis. Location of right eye will be found out by using left eye location.
    desired_left_eye=(0.35, 0.35)
    desired_right_eye_X = 1.0 - desired_left_eye[0]


    # calculating the scale of the aligned image by taking the ratio of the distance between eyes in the "non-aligned input" face-cutout image and "aligned output" 
    # face-cutout image
    current_distance = np.sqrt((dx ** 2) + (dy ** 2))
    desired_distance = (desired_right_eye_X - desired_left_eye[0])
    desired_distance *= 256
    scale = desired_distance / current_distance


    # calculate (x, y)-coordinates of center between the two eyes in the "non-aligned input" face-cutout image
    center_of_eyes = ((left_eye_center_x + right_eye_center_x) // 2, (left_eye_center_y + right_eye_center_y) // 2)


    # calculate the rotation matrix for rotating and scaling the "non-aligned input" face-cutout image into "aligned output" face-cutout image
    M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)


    # update the translation component of the matrix
    tX = desiredFaceWidth * 0.5
    tY = desiredFaceHeight * desiredLeftEye[1]
    M[0, 2] += (tX - eyesCenter[0])
    M[1, 2] += (tY - eyesCenter[1])


    # apply the affine transformation
    (w, h) = (256, 256)
    output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
    

    return(output)


In [ ]:
# this class will help to raise error if user tries to input anything apart from 'yes' or 'no'


class not_ok(Exception):
  pass

In [ ]:
# this function will help to draw-out extracted faces, ask user to recognize his face cutout out of rest all and then will either save or not save a particular 
# face cutout according to user's reply. 


def draw_faces(imagename, result):
  # we will keep noting down names of images iterated once into a notepad file so that later on, those images will not come again
  with open(in_usoon_priv_database+str(id)+"_counter.txt", 'a') as encoded_counter:
    encoded_counter.write(item+"\n") #we will keep noting down names of images iterated once into a notepad file so that later                   
  # load the image via its image path
  data = cv2.imread(imagename)
  # initialize an empty dictionary
  d={}
  # now one-by-one we will iterate through all face cutouts detected by mtcnn in an image.
  # for first run, "i" means first face cutout out of rest face cutouts

  # if you want to see non-aligned face-cutouts then uncomment all lines starting from below............. 
  #for i in range(len(result)):
      #x1, y1, width, height = result[i]['box']
      #x2, y2 = x1 + width, y1 + height
      #face_cutout = data[y1:y2, x1:x2]
      #face_cutout = cv2.resize(face_cutout,(256,256))
      #print(" ")
      #print(" ")
      #next we will refer which face cutout is it. For first iteration it will be 1st photo/face cutout
      #conf = result[i]['confidence']
      #if conf>0.92:
        #pyplot.axis('off')
        #pyplot.imshow(face_cutout)
        #print("Photo numbered",i+1)
        #print("face-cutout before alignment")
        #pyplot.show()
      #else:
        #continue
  #.............upto here


  # this code block will help to show-and-save aligned face-cutouts
  for i in range(len(result)):
    # we will pass values to function "face_alignment" and collect the "aligned face-cutout" in container "face_cutout_aligned"
    face_cutout_aligned = face_alignment(data,result,i)
    print(" ")
    print(" ")
    # we will gave a "unique name" to this aligned face and save this "unique name" of every face cutout as a value with every key "i",i.e. {i: unique_name}
    # this dictionary will be later used to get reference of each face cutout in a particular order while taking user's confirmation and deciding whether to delete 
    # that particular face-cutout or to let it be as it is 
    a=imagename.split('/')[7]
    aa=a.split('.')[0]
    b=company_data+aa+str("=")+str(i)+".jpg" 
    d[i]=b  
    # then we will try to save this face-cutout with its "unique name".
    try:
      cv2.imwrite(d[i], face_cutout_aligned)
    # if failed to save the current face-cutout, we will pick next face-cutout and continue the same process with it
    except Exception:
      continue
    else:
      conf = result[i]['confidence']
      # if current face-cutout gets saved, then next step will be to see "confidence" value corresponding to current face-cutout
      if conf>0.92:
        # If the "confidence" is more than "0.92" then it shows that current face-cutout is of a human. And if that is the case then we will plot that face-cutout
        pyplot.axis('off')
        pyplot.imshow(face_cutout_aligned)
        print("Photo numbered",i+1)
        print("face-cutout after alignment")
        pyplot.show()
      else:
        continue


  for i in range(len(result)):
    # we will only pop-up confirmation question for face-cutouts which without showing error in "try...else" statement (in above code lines), got saved in the database
    if os.path.exists(d[i]):
        while True:
          try:    
              print(" ")
              print(" ")    
              # now we will ask user which face cutout is of him/her           
              confirm = input("Photo numbered "+str(i+1)+" is you? ('1' for 'yes'/ '2' for 'no'/ '3' for 'will resume later on' ):- ")
              for elem in confirm.split():
                  # if user tries to input anything other than '1' (for 'yes') or '2' (for 'no') or '3' (for 'resuming later'), in reply to our confirmation question,
                  # an error will be raised (whose class we have already defined above) saying that you can only reply either 'yes' or 'no'
                  if not (elem=="1" or elem=="2" or elem=="3"):
                    raise not_ok
                  # if user replied to our confirmation question in '1' or '2' or '3', then we will break from here and will go
                  # to 'else' present below 'except'...
                  break
          except not_ok:
              print("Sorry, type in '1' for 'yes'/ '2' for 'no'/ '3' for 'will resume later on'.")
          else:
              #...in this line
              # if user replied '1', we will leave the saved cutout untouched....note:- we have saved face-cutouts earlier via code-lines mentioned above and 
              # now we are deciding whether to move them away from their current destination to non-caring folder named "other", or to leave them there only untouched
              if elem=="1":
                print("saved image")
                print("")
                print("")
              #if user replied '2', then corresponding face-cutout will move to "other" folder   
              elif elem=="2":
                shutil.move(d[i], in_usoon_priv_database+"other")
                print("not saved image") 
                print("")
                print("") 
              else:
                # if user replied '3', then we will delete the latest entry of the face cutout from the notepad ".._counter.txt", so that when user resume the confirmation prcocess he/she
                # at some point of time wuld be able to encounter image on which he left the process in mid-way, thus using available data very carefully without wasting it.
                # After deleting latest entry from notepad ".._counter.txt", we will stop our program
                count=0
                with open(in_usoon_priv_database+str(id)+"_counter.txt", 'r') as encoded_counter:
                  line_list = encoded_counter.readlines()
                  a=len(line_list)
                  with open("/content/drive/My Drive/erevna/Usoon_private_database/"+str(id)+"_counter.txt", 'w') as encoded_counterr:
                    for line in line_list:
                      count=count+1
                      if count!=a:
                        encoded_counterr.writelines(line)
                sys.exit()
              break

In [ ]:
# driver code


# here we are asking user to input his/her user ID 
id = input("please enter your Usoon ID:- ")

# location for company_data containing face cutouts of user
company_data = in_usoon_priv_database+str(id)+"/"
# if there will not be any folder in company_data to store face cutouts of user, then folder will be formed  
if not os.path.exists(company_data):
    os.makedirs(company_data)

# location for company_data containing random pics
company_data_other = in_usoon_priv_database+"other/"
if not os.path.exists(company_data_other):
    os.makedirs(company_data_other)

# location for user_data containing images of user (alone or with someone else) got either from his/her Usoon profile or from face recorder
user_data = os.listdir(out_usoon_priv_database + str(id)+'/pics')

# we will store names of all images iterated till now in list called 'names'
try:
  names=[]
  with open(in_usoon_priv_database+str(id)+"_counter.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
      linee=line.split('/')[7]
      lineee=linee.split('\n')[0]
      names.append(lineee)
except Exception:
  names=[]


a=1
# we take first image of user from 'user_data' 
for entry in user_data:
  # just pick-out name (without '.jpg')
  entryyy = entry.split('.')[0]
  # make full path of image (came from using 'for loop' on 'user_data')
  item =out_usoon_priv_database+str(id)+'/pics/'+ str(entry)
  # load that image from 'user_data'
  image_loadedd = cv2.imread(item)
  # create the MTCNN detector, using default weights and detect faces in images
  face_detector = MTCNN()
  faces = face_detector.detect_faces(image_loadedd)
  if a<2 and len(faces)<5 and len(faces)>0 and entry not in names:
    if len(faces)<2:
      continue
    else:
      a=4
      draw_faces(item, faces)
  elif a>=2 and len(faces)<5 and len(faces)>0 and entry not in names:
    draw_faces(item, faces)

To see number of face-cutouts in 'parent' folder of each user

In [ ]:
for id in ids:
  c = os.listdir(in_usoon_priv_database+str(id))
  print("number of face-cutouts in "+str(id)+"'s parent folder are:"+str(len(c)))

To **split** processed face-cutouts of user from company's database into **validation dataset** and **test dataset**.  



In [ ]:
# function that will split face cutouts of an user into validation dataset and test dataset
def split(id):
    # load the face-cutouts of an user from 'company_data' 
    company_data = os.listdir(in_usoon_priv_database+str(id))


    # create path of 'company_data_train' folder where some face-cutouts will be stored for 'training' purpose   
    company_data_train = in_usoon_priv_database+str(id)+"_train"
    if not os.path.exists(company_data_train):
        os.makedirs(company_data_train)


    # create path of 'company_data_val' folder where some face-cutouts will be stored for 'testing' purpose   
    company_data_val = in_usoon_priv_database+str(id)+"_val"
    if not os.path.exists(company_data_val):
        os.makedirs(company_data_val)


    # these containers will store names of all images stored in 'company_data_val' and 'company_data_train'
    company_data_vall = os.listdir(in_usoon_priv_database+str(id)+"_val/")
    company_data_trainn = os.listdir(in_usoon_priv_database+str(id)+"_train/")


    # this variable will be used later
    count=0


    # we will pick finalised face-cutouts and save first 30s for validation process and rest all for training process
    for x in company_data:
      xx=in_usoon_priv_database+id+"/"+str(x)
      count=count+1
      if count<31:
        shutil.move(xx, in_usoon_priv_database+str(id)+"_val")    
      else:
        shutil.move(xx, in_usoon_priv_database+str(id)+"_train")




# driver code
for id in ids:
    split(id)

To see number of face-cutouts in 'train' and 'val' folder of each user.

In [ ]:
for id in ids:
  for mode in modes:
    c = os.listdir(in_usoon_priv_database+str(id)+"_"+str(mode))
    print("number of face-cutouts in "+str(id)+"'s "+str(mode)+" folder are:"+str(len(c)))

To **undo** above action and proof:- all 'train' and 'val' folders will be empty

In [ ]:
for id in ids:
  for mode in modes:
    a = os.listdir(in_usoon_priv_database+str(id)+"_"+str(mode))
    b = in_usoon_priv_database+str(id)

    for x in a:
      xx=in_usoon_priv_database+str(id)+"_"+str(mode)+"/"+str(x) 
      shutil.move(xx, b)


for id in ids:
  for mode in modes:
    c = os.listdir(in_usoon_priv_database+str(id)+"_"+str(mode))
    print("number of face-cutouts in "+str(id)+"'s "+str(mode)+" folder are:"+str(len(c)))

To convert processed train and validation face-cutouts datasets into corresponding **numpy** and then compressing it all together.

In [ ]:
# this function firt "resize" and then "arrays" every single face-cutouts present in "renuka_train","renuka_val","malika_train" and "malika_val" folders 
# then save each folders'(mentioned above)resized&arrayed elements into two lists and return these two lists, when called four times 


def arrays(mode,id):
  face_numpy = list()
  faces = list()
  label = list()
  company_data = os.listdir(in_usoon_priv_database+str(id)+"_"+str(mode))
  for x in company_data:
    xx=in_usoon_priv_database+str(id)+"_"+str(mode)+"/"+str(x)
    image_loaded = cv2.imread(xx)
    try:
      image_loadedd = cv2.resize(image_loaded,(160, 160))
    except Exception:
      continue
    face_cutout = asarray(image_loadedd)
    faces.append(face_cutout)
    label.append(id)
  return faces,label




d={}
# we are converting all face-cutouts into array form. 
# after that we keep all "arrayed training face-cutouts", all "arrayed validation face-cutouts",
# all "labels corresponding to training face-cutouts" and all "labels corresponding to validation face-cutouts"
for mode in modes:
  # we take a particular "mode" (like "train") and initialise two lists "numpy_face" and "numpy_label".
  # thus for "train" there will be different "numpy_face" and "numpy_label" lists....and seperate for "val" mode.
  numpy_face = list()
  numpy_label = list()
  # now we call all "training face-cutouts" realted to first id   
  for id in ids:
    # call the function "arrays", which will return "all arrayed training face-cutouts" list "faces" and "all labels 
    # corresponding to training face-cutouts" list "label" in forms "numpy_faces" and "numpy_labels" respectively 
    numpyy_faces,numpyy_labels = arrays(mode,id)
    # next we add list "numpyy_faces" (containing "all arrayed training face-cutouts") into "numpy_face" list and 
    #"numpyy_labels" (containing "all labels corresponding to training face-cutouts") into "numpy_label". 
    # Later in these senior-most lists, "all arrayed validation face-cutouts" and "all labels corresponding to validation
    # face-cutouts" will be added.
    numpy_face.extend(numpyy_faces)
    numpy_label.extend(numpyy_labels)

  # just arraying senior-most lists "numpy_face" and 'numpy_label" 
  numpy_faces = asarray(numpy_face)
  numpy_labels = asarray(numpy_label)
  numpy_face_key="numpy_face_"+str(mode)
  numpy_label_key="numpy_label_"+str(mode)
  # and then these "arrayed senior-most lists" will be saved as "value" corresponding to dictionary keys of 
  # "numpy_face_train", "numpy_face_val", "numpy_label_train" and "numpy_label_val"
  d[numpy_face_key]=numpy_faces
  d[numpy_label_key]=numpy_labels


print("Shape of numpy_face_train is:- "+str(d['numpy_face_train'].shape));print("Shape of numpy_label_train is:- "+str(d['numpy_label_train'].shape));print("Shape of numpy_face_val is:- "+str(d['numpy_face_val'].shape));print("Shape of numpy_label_val is:- "+str(d['numpy_label_val'].shape))  
filename_for_saving = in_usoon_priv_database+"numpy_compressed_dataset.npz"
# then we save these four lists by calling them through corresponding dictionary key.
savez_compressed(filename_for_saving, d['numpy_face_train'], d['numpy_label_train'], d['numpy_face_val'], d['numpy_label_val'])

Load back the numpy compressed dataset, convert to **facenet-embeddings**, again compress and save


In [ ]:
# takes "arrayed training and validation face-cutouts" and return back corresponding embeddings to "collect_embedding"


def numpy_to_embedding(model, numpy_array):
  numpy_array = numpy_array.astype('float32')
  mean, std = numpy_array.mean(), numpy_array.std()
  numpy_array = (numpy_array - mean) / std
  numpy_array = expand_dims(numpy_array, axis=0)
  yhat = model.predict(numpy_array)
  face_embedding=yhat[0]
  return face_embedding


# collect embeddings of firstly "arrayed training face-cutouts" from "numpy_to_embedding", append in list "container", return that list to 
# "driver code"......then do same for "arrayed validation face-cutouts"
def collect_embedding(model,numpy_array):
  container=list()
  a=bag[numpy_array]
  for b in a:
    embedding = numpy_to_embedding(model,b)
    container.append(embedding)
    containerr = asarray(container)
  return containerr




model = load_model(out_usoon_priv_database+'facenet_keras.h5'
# we load back the "arrayed dataset" into 4 containers and save them as "values" to cooresponding "keys" in "bag" dictionary
data = load(in_usoon_priv_database+"numpy_compressed_dataset.npz")
numpy_face_train, numpy_label_train, numpy_face_val, numpy_label_val = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# print("numpy_face_train_shape:- ",numpy_face_train.shape);print("numpy_label_train_shape:- ",numpy_label_train.shape);print("numpy_face_val_shape:- ",numpy_face_val.shape);print("numpy_label_val_shape:-  ",numpy_label_val.shape) 
bag={}
bag['numpy_face_train']=numpy_face_train
bag['numpy_face_val']=numpy_face_val
bag['numpy_label_train']=numpy_label_train
bag['numpy_label_val']=numpy_label_val

bucket={}
listtt=['face_train', 'face_val']
# now we call "collect_embedding" function, who will return "embeddings of all training face-cutouts"
# and "embeddings of all validation face-cutouts". We will save these returned embeddings into "bucket" dictionary as "value"
# to corresponding "keys"
for name in listtt:
  namee="numpy_"+name
  bucket_key = "embed_"+name
  bucket[bucket_key]=collect_embedding(model, namee)


# next we pick-up training&validation face-cutouts embeddings from dictionary "bucket" and save them into "embed_face_train" 
# and "embed_face_val".....and pick-up training&validation face-cutouts labels from dictionary "bag" and save them into 
# "embed_label_train" and "embed_label_val"
embed_face_train=bucket['embed_face_train']
embed_face_val=bucket['embed_face_val']
embed_label_train=bag['numpy_label_train']
embed_label_val=bag['numpy_label_val']
print(" ")
print(" ")
print('embed_face_train_shape:- ',embed_face_train.shape);print('embed_label_train_shape:- ',embed_label_train.shape);print('embed_face_val_shape:- ',embed_face_val.shape);print('embed_label_val_shape:- ',embed_label_val.shape)


# then save "training&validation face cutouts embeddings" and "training&validation face cutouts labels"
filename_for_saving = in_usoon_priv_database+"embedding_compressed_dataset.npz"
savez_compressed(filename_for_saving, embed_face_train, embed_label_train, embed_face_val, embed_label_val)

# **Training & Validating**

**Implementating SVM** on embedded test and train datasets

In [ ]:
data = load(in_usoon_priv_database+'embedding_compressed_dataset.npz')
embed_face_train, embed_label_train, embed_face_val, embed_label_val = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# print('embed_face_train_shape',embed_face_train.shape);print('embed_label_train_shape',embed_label_train.shape);print('embed_face_val_shape',embed_face_val.shape);print('embed_label_val_shape',embed_label_val.shape)
# normalize arrayed face-cutouts
in_encoder = Normalizer(norm='l2')
embed_face_trainn = in_encoder.transform(embed_face_train)
embed_face_vall = in_encoder.transform(embed_face_val)
# label encoding and saving this "tool" for future use
out_encoder = LabelEncoder()
out_encoder.fit(embed_label_train)
embed_label_trainn = out_encoder.transform(embed_label_train)
embed_label_vall = out_encoder.transform(embed_label_val)
numpy.save(in_usoon_priv_database+'classes.npy', out_encoder.classes_)


# training the model and saving it for future use
model = SVC(kernel='linear', probability=True)
model.fit(embed_face_trainn, embed_label_trainn)
with open(in_usoon_priv_database+"model.pkl", "wb") as encoded_pickle:
    pickle.dump(model, encoded_pickle)


# using the trained model to predict labels
yhat_train = model.predict(embed_face_trainn)
yhat_val = model.predict(embed_face_vall)
# calculating accuracy score and printing it
score_train = accuracy_score(embed_label_trainn, yhat_train)
score_val = accuracy_score(embed_label_vall, yhat_val)
print(" ")
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_val*100))

 
Accuracy: train=98.431, test=97.778


# **Testing**

In [ ]:
idd="renuka"

In [ ]:
def draw_and_numpy_faces(data, result, imagename):
  d={}
  face_numpy=list()
  face_dimension=list()
  face_id=list()
  face_score=list()

  for i in range(len(result)):
      x1, y1, width, height = result[i]['box']
      score=result[i]['confidence']
      x2, y2 = x1 + width, y1 + height
      imagee = data[y1:y2, x1:x2]
      face_cutout = cv2.resize(imagee,(160, 160))   
      face_cutout_array = asarray(face_cutout)
      face_numpy.append(face_cutout_array)
      face_dimension.append(result[i]['box'])
      face_id.append(imagename)
      face_score.append(score)
  return face_numpy,face_dimension,face_id,face_score


def numpyy(test_data):
    face_numpy_collect=list()
    face_id_collect=list()
    face_dimension_collect=list()
    face_score_collect=list()

    for entry in test_data:
      entryyy = entry.split('.')[0]
      item = out_usoon_priv_database+'test_pics/'+ str(entry)
      image_loadedd = cv2.imread(item)
      face_detector = MTCNN()
      faces = face_detector.detect_faces(image_loadedd)
      if len(faces)<5 and len(faces)>0 :
        face_numpy,face_dimension,face_id,face_score = draw_and_numpy_faces(image_loadedd, faces, item)
        face_numpy_collect.extend(face_numpy)
        face_dimension_collect.extend(face_dimension)
        face_id_collect.extend(face_id)
        face_score_collect.extend(face_score)
    face_numpy_collect = asarray(face_numpy_collect)
    face_dimension_collect = asarray(face_dimension_collect)
    face_id_collect = asarray(face_id_collect)
    face_score_collect = asarray(face_score_collect)
    print("numpy_face_train_shape:- ",face_numpy_collect.shape)
    return face_numpy_collect,face_dimension_collect,face_id_collect,face_score_collect




#location for test_data containing whole image of user (alone or with someone else)
test_data = os.listdir(out_usoon_priv_database+'test_pics/')
numpyy_pic,face_dimension_collect,face_id_collect,face_score_collect=numpyy(test_data)

In [ ]:
def numpy_to_embedding(model, numpy_array):
  numpy_array = numpy_array.astype('float32')
  mean, std = numpy_array.mean(), numpy_array.std()
  numpy_array = (numpy_array - mean) / std
  numpy_array = expand_dims(numpy_array, axis=0)
  yhat = model.predict(numpy_array)
  face_embedding=yhat[0]
  return face_embedding


def collect_embedding(model,numpy_array):
  container=list()
  for b in numpy_array:
    embedding = numpy_to_embedding(model,b)
    container.append(embedding)
    containerr = asarray(container)
  return containerr




model = load_model(out_usoon_priv_database+'facenet_keras.h5')
embed_face_test=collect_embedding(model, numpyy_pic)
print('embed_face_test_shape:- ',embed_face_test.shape)

embed_face_test_shape:-  (15, 128)


In [ ]:
in_encoder = Normalizer(norm='l2')
embed_face_test = in_encoder.transform(embed_face_test)


out_encoder = LabelEncoder()
out_encoder.classes_ = numpy.load(in_usoon_priv_database+'classes.npy')


filename = in_usoon_priv_database+"model.pkl"
model = pickle.load(open(filename, 'rb'))


for i in range(embed_face_test.shape[0]):
    random_face_emb = embed_face_test[i]
    samples = expand_dims(random_face_emb, axis=0)
    yhat_class = model.predict(samples)
    class_index = yhat_class[0]
    predict_names = out_encoder.inverse_transform(yhat_class)

    if predict_names[0]!=idd and predict_names[0]!="other":
      score=face_score_collect[i]
      print('Predicted: %s' % predict_names[0])
      print(score)
      test_data_singular=face_id_collect[i]
      x1, y1, width, height=face_dimension_collect[i]
      image_loadedd = cv2.imread(test_data_singular)
      pyplot.axis('off')
      pyplot.imshow(image_loadedd)
      ax = pyplot.gca()
      rect = Rectangle((x1, y1), width, height, fill=False, color='red')
      ax.add_patch(rect)
      # show the plot
      pyplot.show()
